In [ ]:
!pip install opencv-python
!apt-get install tesseract-ocr tesseract-ocr-heb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  tesseract-ocr-heb
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 432 kB of archives.
After this operation, 976 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-heb all 1:4.00~git30-7274cfa-1.1 [432 kB]
Fetched 432 kB in 1s (704 kB/s)
Selecting previously unselected package tesseract-ocr-heb.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-heb_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-heb (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-heb (1:4.00~git30-7274cfa-1.1) ...


In [ ]:
import cv2
import os
import shutil
from tqdm import tqdm
import numpy as np

In [ ]:
# PATHS
SOURCE_IMAGES = "/content/drive/MyDrive/Miki_class/Project/Catalog/images"
SINGLE_AD_DEST = "/content/drive/MyDrive/Miki_class/Project/Catalog/classified_images/single_ads"
MULTI_AD_DEST = "/content/drive/MyDrive/Miki_class/Project/Catalog/classified_images/multi_ads"

# Create destination folders
os.makedirs(SINGLE_AD_DEST, exist_ok=True)
os.makedirs(MULTI_AD_DEST, exist_ok=True)

In [ ]:
# Count how many ad regions are detected in an image
def count_ad_regions(image_path):
  try:
    # Load image
    image = cv2.imread(image_path)
    if image is None:
      return 0

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Edge detection
    edges = cv2.Canny(gray, 50, 120)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Count valid ad regions
    valid_regions = 0
    image_area = image.shape[0] * image.shape[1]
    min_area = image_area / 80  # Lenient threshold
    min_size = 150

    for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      area = w * h

      # Count if meets minimum criteria
      if w >= min_size and h >= min_size and area >= min_area:
        aspect_ratio = w / h
        if 0.1 <= aspect_ratio <= 10.0:  # Very lenient aspect ratio
          valid_regions += 1

    return valid_regions

  except Exception as e:
    print(f"Error processing {image_path}: {e}")
    return 0

In [ ]:
def classify_image(image_path):
  ad_count = count_ad_regions(image_path)

  # Classification logic
  if ad_count >= 4:
    return 'multi'
  elif ad_count <= 1:
    return 'single'
  else:
    # Default to multi to be safe
    return 'multi'

In [ ]:
def auto_classify_all_images():

  if not os.path.exists(SOURCE_IMAGES):
    print(f" Source directory not found: {SOURCE_IMAGES}")
    return

  # Get all issue folders
  issue_folders = [f for f in os.listdir(SOURCE_IMAGES)
                  if os.path.isdir(os.path.join(SOURCE_IMAGES, f))]

  print(f"📁 Found {len(issue_folders)} issue folders to process")

  # Statistics
  stats = {
      'total_images': 0,
      'single_ads': 0,
      'multi_ads': 0,
      'errors': 0
  }

  # Process each issue folder
  for issue_folder in tqdm(issue_folders, desc="Processing issues"):
    issue_path = os.path.join(SOURCE_IMAGES, issue_folder)

    # Create destination folders for this issue
    single_issue_dest = os.path.join(SINGLE_AD_DEST, issue_folder)
    multi_issue_dest = os.path.join(MULTI_AD_DEST, issue_folder)
    os.makedirs(single_issue_dest, exist_ok=True)
    os.makedirs(multi_issue_dest, exist_ok=True)

    # Get all images in this issue
    images = [f for f in os.listdir(issue_path)
              if f.endswith(('.jpg', '.jpeg', '.png'))]

    print(f"\n📖 Issue {issue_folder}: {len(images)} images")

    # Process each image
    for image_filename in tqdm(images, desc=f"Issue {issue_folder}", leave=False):
      image_path = os.path.join(issue_path, image_filename)

      try:
        # Classify the image
        classification = classify_image(image_path)
        stats['total_images'] += 1

        # Copy to appropriate destination
        if classification == 'single':
          dest_path = os.path.join(single_issue_dest, image_filename)
          stats['single_ads'] += 1
        # multi
        else:
          dest_path = os.path.join(multi_issue_dest, image_filename)
          stats['multi_ads'] += 1

        # Copy file
        shutil.copy2(image_path, dest_path)

      except Exception as e:
        print(f" Error processing {image_filename}: {e}")
        stats['errors'] += 1

    # Print progress for this issue
    issue_single = len([f for f in os.listdir(single_issue_dest) if f.endswith(('.jpg', '.jpeg', '.png'))])
    issue_multi = len([f for f in os.listdir(multi_issue_dest) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"   Issue {issue_folder}: {issue_single} single, {issue_multi} multi")

  return stats

In [ ]:
print(" Starting automatic classification of all images")

classification_stats = auto_classify_all_images()
# Final summary
print(f" FINAL STATISTICS:")
print(f"  Total images processed: {classification_stats['total_images']}")
print(f"  Single-ad pages: {classification_stats['single_ads']}")
print(f"  Multi-ad pages: {classification_stats['multi_ads']}")
print(f"  Errors: {classification_stats['errors']}")
print(f"  Success rate: {((classification_stats['total_images'] - classification_stats['errors']) / classification_stats['total_images'] * 100):.1f}%")

print(f"\n📂 Results saved to:")
print(f"  Single ads: {SINGLE_AD_DEST}")
print(f"  Multi ads: {MULTI_AD_DEST}")

 Starting automatic classification of all images
📁 Found 29 issue folders to process


Processing issues:   0%|          | 0/29 [00:00<?, ?it/s]


📖 Issue 1216: 195 images



Processing issues:   3%|▎         | 1/29 [00:26<12:14, 26.21s/it]

   Issue 1216: 42 single, 153 multi

📖 Issue 1217: 144 images


...

   Issue 1229: 27 single, 133 multi

📖 Issue 1230: 208 images



Processing issues:  97%|█████████▋| 28/29 [10:05<00:21, 21.75s/it]

   Issue 1230: 38 single, 170 multi

📖 Issue 1231: 176 images



Processing issues: 100%|██████████| 29/29 [10:21<00:00, 21.42s/it]

   Issue 1231: 32 single, 144 multi
 FINAL STATISTICS:
  Total images processed: 5171
  Single-ad pages: 954
  Multi-ad pages: 4217
  Errors: 0
  Success rate: 100.0%

📂 Results saved to:
  Single ads: /content/drive/MyDrive/Miki_class/Project/Catalog/classified_images/single_ads
  Multi ads: /content/drive/MyDrive/Miki_class/Project/Catalog/classified_images/multi_ads
